In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime as dt
import random
from sportsreference.ncaab.teams import Teams
from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from matplotlib import pyplot

In [2]:
teams_df = pd.DataFrame()
i = 2010
while i <=2018:
    print("Downloading data for the", i, "season.")
    start = dt.datetime.now()
    teams = Teams(year = i)
    end = dt.datetime.now()
    teams_temp = teams.dataframes
    teams_temp['Season'] = i
    teams_df = pd.concat([teams_df, teams_temp])
    span = end - start
    print(i, "took", span, "seconds to download.")
    i += 1

2010 took 0:00:28.001999 seconds to download.
2011 took 0:00:28.644882 seconds to download.
2012 took 0:00:28.214825 seconds to download.
2013 took 0:00:31.007702 seconds to download.
2014 took 0:00:27.818103 seconds to download.
2015 took 0:00:25.993688 seconds to download.
2016 took 0:00:33.348126 seconds to download.
2017 took 0:00:26.943783 seconds to download.
2018 took 0:00:27.643593 seconds to download.


In [3]:
teams_df.head(10)

,abbreviation,assist_percentage,assists,away_losses,away_wins,block_percentage,blocks,conference,conference_losses,conference_wins,...,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_field_goal_attempts,two_point_field_goal_percentage,two_point_field_goals,win_percentage,wins,Season
AIR-FORCE,AIR-FORCE,61.6,389,10,0,6.2,51,mwc,15,1,...,851,0.529,19.0,394,866,0.527,456,0.323,10,2010
AKRON,AKRON,53.9,473,4,7,8.5,114,mac,4,12,...,1269,0.521,16.4,464,1330,0.483,642,0.686,24,2010
ALABAMA-AM,ALABAMA-AM,48.1,288,9,3,12.7,143,swac,10,8,...,1004,0.463,18.8,444,1197,0.410,491,0.407,11,2010
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,51.1,390,4,9,7.3,91,cusa,5,11,...,1248,0.518,17.0,451,1237,0.474,586,0.735,25,2010
ALABAMA-STATE,ALABAMA-STATE,60.0,400,11,6,11.1,129,swac,6,12,...,1101,0.499,20.0,501,1064,0.448,477,0.516,16,2010
ALABAMA,ALABAMA,54.2,429,6,3,10.9,127,sec,10,6,...,1126,0.525,16.4,410,1294,0.476,616,0.531,17,2010
ALBANY-NY,ALBANY-NY,53.5,380,15,3,8.2,97,america-east,14,2,...,1156,0.503,20.6,517,1151,0.448,516,0.219,7,2010
ALCORN-STATE,ALCORN-STATE,41.3,269,18,0,7.4,98,swac,16,2,...,1065,0.456,23.7,642,1280,0.403,516,0.065,2,2010
AMERICAN,AMERICAN,65.6,447,12,4,6.6,81,patriot,7,7,...,1097,0.518,19.6,458,1069,0.465,497,0.355,11,2010
APPALACHIAN-STATE,APPALACHIAN-STATE,46.9,447,8,8,7.7,102,southern,5,13,...,1424,0.581,18.6,544,1303,0.518,675,0.649,24,2010


In [6]:
games = pd.read_csv(r'data/TourneyCompactResults.csv')

In [7]:
games = games[['Season','WTeamID','LTeamID']]
games['Winner'] = games['WTeamID']
games = games.rename(columns = {'WTeamID' : 'Team_A', 'LTeamID': 'Team_B'})
games.head()

,Season,Team_A,Team_B,Winner
0,1985,1116,1234,1116
1,1985,1120,1345,1120
2,1985,1207,1250,1207
3,1985,1229,1425,1229
4,1985,1242,1325,1242


In [8]:
i = 0
games['Team_X'] = 0
games['Team_Y'] = 0
while i < len(games):
    games['Team_X'][i] = random.choice(([games['Team_A'][i],games['Team_B'][i]]))
    i += 1
games['Team_Y'] = np.where(games['Team_X'] == games['Team_A'], games['Team_B'], games['Team_A'])
games['Result'] = np.where(games['Winner'] == games['Team_X'], 'X', 'Y')
games = games.drop(columns = ['Team_A','Team_B'], axis = 1)

In [9]:
games.head(25)

,Season,Winner,Team_X,Team_Y,Result
0,1985,1116,1234,1116,Y
1,1985,1120,1345,1120,Y
2,1985,1207,1207,1250,X
3,1985,1229,1425,1229,Y
4,1985,1242,1325,1242,Y
5,1985,1246,1449,1246,Y
6,1985,1256,1338,1256,Y
7,1985,1260,1233,1260,Y
8,1985,1314,1292,1314,Y
9,1985,1323,1333,1323,Y


In [72]:
team_data = teams_df[['Season','abbreviation','assist_percentage','block_percentage','effective_field_goal_percentage','field_goal_percentage','free_throw_attempt_rate','free_throw_percentage','net_rating','offensive_rating','offensive_rebound_percentage','pace','simple_rating_system','steal_percentage','strength_of_schedule','three_point_attempt_rate','three_point_field_goal_percentage','total_rebound_percentage','true_shooting_percentage','turnover_percentage','two_point_field_goal_percentage','win_percentage']]

In [73]:
games = games.loc[games['Season'] >= 2010]
games.head()

,Season,Winner,Team_X,Team_Y,Result
1584,2010,1115,1115,1457,X
1585,2010,1124,1358,1124,Y
1586,2010,1139,1431,1139,Y
1587,2010,1140,1196,1140,Y
1588,2010,1242,1242,1250,X


In [74]:
team_abb = pd.read_csv(r'data/team_names.csv')

In [75]:
team_abb = team_abb[['abbreviation','Team_Id']]

In [76]:
team_data_abb = team_data.merge(team_abb, left_on = 'abbreviation', right_on = 'abbreviation', how = 'left').dropna()
team_data_abb['Team_Id'] = team_data_abb['Team_Id'].astype(np.int64)

In [77]:
team_data_abb.head()

,Season,abbreviation,assist_percentage,block_percentage,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,net_rating,offensive_rating,...,steal_percentage,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_percentage,total_rebound_percentage,true_shooting_percentage,turnover_percentage,two_point_field_goal_percentage,win_percentage,Team_Id
0,2010,AIR-FORCE,61.6,6.2,0.504,0.443,0.367,0.635,-10.0,96.0,...,8.4,3.13,0.394,0.313,46.8,0.529,19.0,0.527,0.323,1102
1,2010,AKRON,53.9,8.5,0.491,0.433,0.363,0.657,7.6,104.8,...,9.0,-1.50,0.343,0.339,51.6,0.521,16.4,0.483,0.686,1103
2,2010,ALABAMA-AM,48.1,12.7,0.416,0.382,0.474,0.635,-5.2,89.5,...,12.5,-13.71,0.237,0.291,46.8,0.463,18.8,0.410,0.407,1105
3,2010,ALABAMA-BIRMINGHAM,51.1,7.3,0.471,0.422,0.457,0.694,10.0,102.4,...,10.0,2.90,0.315,0.311,53.6,0.518,17.0,0.474,0.735,1412
4,2010,ALABAMA-STATE,60.0,11.1,0.462,0.404,0.448,0.641,-2.1,94.7,...,10.8,-12.02,0.356,0.324,51.3,0.499,20.0,0.448,0.516,1106


In [78]:
games_a = games.merge(team_data_abb, left_on = ['Team_X','Season'], right_on = ['Team_Id','Season'], how = 'left')
games_b = games_a.merge(team_data_abb, left_on = ['Team_Y','Season'], right_on = ['Team_Id','Season'], how = 'left')
games_b = games_b.dropna()
print(games_b.head(15))

    Season  Winner  Team_X  Team_Y Result        abbreviation_x  \
0     2010    1115    1115    1457      X   ARKANSAS-PINE-BLUFF   
1     2010    1124    1358    1124      Y     SAM-HOUSTON-STATE   
2     2010    1139    1431    1139      Y         TEXAS-EL-PASO   
3     2010    1140    1196    1140      Y               FLORIDA   
4     2010    1242    1242    1250      X                KANSAS   
5     2010    1243    1317    1243      Y           NORTH-TEXAS   
6     2010    1246    1190    1246      Y  EAST-TENNESSEE-STATE   
7     2010    1293    1435    1293      Y            VANDERBILT   
8     2010    1307    1307    1285      X            NEW-MEXICO   
9     2010    1320    1424    1320      Y      NEVADA-LAS-VEGAS   
10    2010    1325    1207    1325      Y            GEORGETOWN   
11    2010    1330    1330    1323      X          OLD-DOMINION   
12    2010    1388    1388    1350      X        SAINT-MARYS-CA   
13    2010    1397    1361    1397      Y       SAN-DIEGO-STAT

In [79]:
games_b['Winner'] = np.where(games_b['Winner'] == games_b['Team_X'], games_b['abbreviation_x'], games_b['abbreviation_y'])
games_b['Team_X'] = games_b['abbreviation_x']
games_b['Team_Y'] = games_b['abbreviation_y']

In [80]:
ml_input = games_b.drop(columns = ['Season','Winner','Team_X','Team_Y','abbreviation_x','abbreviation_y','Team_Id_x','Team_Id_y','win_percentage_x','win_percentage_y'], axis = 1)

In [81]:
X_all = ml_input.drop(['Result'],1)
y_all = ml_input['Result']

In [82]:
cols = [['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y']]
for col in cols:
    X_all[col] = scale(X_all[col])

In [83]:
len(X_all)

533

In [131]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                   stratify = y_all)

In [128]:
#for measuring training time
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    time_taken = end - start
    
    # Print the results
    print('Trained model', clf.__class__.__name__, 'in', time_taken, ' seconds')

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    time_taken = end - start
    print("Made predictions in", time_taken, "seconds.")
    
    return f1_score(target, y_pred, pos_label='X'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    #print("Training something using a training set size of %d. . .").format(len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print("F1 score and accuracy score for training set:", f1, " , ", acc)
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set:", f1, " , ", acc)

In [132]:
clf_A = LogisticRegression(random_state = 64)
clf_B = SVC(random_state = 64, kernel = 'rbf')
clf_C = xgb.XGBClassifier(seed = 64)

train_predict(clf_A, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_B, X_train, y_train, X_test, y_test)
print('')
train_predict(clf_C, X_train, y_train, X_test, y_test)
print('')

Trained model LogisticRegression in 0.005012989044189453  seconds
Made predictions in 0.0020177364349365234 seconds.
F1 score and accuracy score for training set: 0.7401574803149606  ,  0.7518796992481203
Made predictions in 0.0010042190551757812 seconds.
F1 score and accuracy score for test set: 0.6875  ,  0.7014925373134329

Trained model SVC in 0.014036893844604492  seconds
Made predictions in 0.008023262023925781 seconds.
F1 score and accuracy score for training set: 0.8632707774798928  ,  0.8721804511278195
Made predictions in 0.003007650375366211 seconds.
F1 score and accuracy score for test set: 0.694214876033058  ,  0.7238805970149254

Trained model XGBClassifier in 0.15441298484802246  seconds
Made predictions in 0.004010200500488281 seconds.
F1 score and accuracy score for training set: 0.9820051413881747  ,  0.9824561403508771
Made predictions in 0.0030083656311035156 seconds.
F1 score and accuracy score for test set: 0.6875  ,  0.7014925373134329



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [133]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer


# TODO: Create the parameters list you wish to tune
parameters = { 'learning_rate' : [0.1],
               'n_estimators' : [40],
               'max_depth': [3],
               'min_child_weight': [3],
               'gamma':[0.4],
               'subsample' : [0.8],
               'colsample_bytree' : [0.8],
               'scale_pos_weight' : [1],
               'reg_alpha':[1e-5]
             }  

# TODO: Initialize the classifier
clf = xgb.XGBClassifier(seed=2)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,pos_label='X')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,
                        scoring=f1_scorer,
                        param_grid=parameters,
                        cv=5)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
f1, acc = predict_labels(clf, X_train, y_train)
print("F1 score and accuracy score for training set:", f1, " , ", acc)
    
f1, acc = predict_labels(clf, X_test, y_test)
print("F1 score and accuracy score for test set:", f1, " , ", acc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.4, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=3, missing=None,
       n_estimators=40, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=1e-05,
       reg_lambda=1, scale_pos_weight=1, seed=2, silent=True,
       subsample=0.8)
Made predictions in 0.006488323211669922 seconds.
F1 score and accuracy score for training set: 0.873015873015873  ,  0.8796992481203008
Made predictions in 0.0005013942718505859 seconds.
F1 score and accuracy score for test set: 0.7131782945736435  ,  0.7238805970149254


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [98]:
X_all.head()

,assist_percentage_x,block_percentage_x,effective_field_goal_percentage_x,field_goal_percentage_x,free_throw_attempt_rate_x,free_throw_percentage_x,net_rating_x,offensive_rating_x,offensive_rebound_percentage_x,pace_x,...,pace_y,simple_rating_system_y,steal_percentage_y,strength_of_schedule_y,three_point_attempt_rate_y,three_point_field_goal_percentage_y,total_rebound_percentage_y,true_shooting_percentage_y,turnover_percentage_y,two_point_field_goal_percentage_y
0,0.739910,-0.181037,-2.560138,-2.220221,2.092906,-1.195040,-2.217333,-3.203292,0.542612,0.199613,...,-0.023783,-3.104573,1.561782,-2.790766,-1.660931,-4.160374,-0.337375,-4.246506,-0.058353,-2.988560
1,4.018267,-1.286182,0.621306,0.186530,-0.237394,-0.162839,0.145619,0.341336,0.362594,1.194525,...,0.139153,0.439975,-0.155825,0.410307,-0.238350,0.890496,1.006636,1.045233,1.147160,1.005588
2,0.059496,-0.181037,0.335334,0.667881,0.294078,-1.047583,0.318902,-0.602614,-1.180421,0.945797,...,-0.871050,-0.038838,0.509055,-0.205284,1.335110,-0.694814,0.097452,0.183787,0.664955,0.025206
3,-0.146690,-1.217111,-0.880050,-0.644893,-0.973279,-0.162839,-0.720796,-0.448500,0.902649,-0.080207,...,1.377465,0.553028,1.063122,-0.639328,-0.130579,2.070261,0.453220,1.701573,-0.600834,0.279379
4,0.925477,1.822038,1.157505,1.324268,0.641578,-0.280805,1.846945,1.362343,1.262686,0.821433,...,1.051594,-2.393003,-0.709892,-2.890930,-0.841869,1.406643,-0.376904,0.019703,0.544403,-1.100418


In [93]:
teams_19 = Teams(year = 2019)
teams_df_19 = teams.dataframes
teams_df_19 = teams_df_19[['abbreviation','assist_percentage','block_percentage','effective_field_goal_percentage','field_goal_percentage','free_throw_attempt_rate','free_throw_percentage','free_throws_per_field_goal_attempt','net_rating','offensive_rating','offensive_rebound_percentage','opp_assist_percentage','opp_block_percentage','opp_effective_field_goal_percentage','opp_field_goal_percentage','opp_free_throw_attempt_rate','opp_free_throw_percentage','opp_free_throws_per_field_goal_attempt','opp_offensive_rating','opp_offensive_rebound_percentage','opp_steal_percentage','opp_three_point_attempt_rate','opp_three_point_field_goal_percentage','opp_total_rebound_percentage','opp_true_shooting_percentage','opp_two_point_field_goal_percentage','pace','simple_rating_system','steal_percentage','strength_of_schedule','three_point_attempt_rate','three_point_field_goal_percentage','total_rebound_percentage','true_shooting_percentage','turnover_percentage','two_point_field_goal_percentage']]

In [99]:
teams_df_19.head()

,abbreviation,assist_percentage,block_percentage,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,free_throws_per_field_goal_attempt,net_rating,offensive_rating,...,pace,simple_rating_system,steal_percentage,strength_of_schedule,three_point_attempt_rate,three_point_field_goal_percentage,total_rebound_percentage,true_shooting_percentage,turnover_percentage,two_point_field_goal_percentage
ABILENE-CHRISTIAN,ABILENE-CHRISTIAN,55.4,11.6,0.521,0.464,0.309,0.701,0.217,3.5,102.2,...,71.6,-9.14,11.3,-6.82,0.350,0.325,49.7,0.549,17.7,0.539
AIR-FORCE,AIR-FORCE,60.7,8.1,0.490,0.419,0.318,0.734,0.233,-5.7,100.8,...,67.7,-4.31,9.5,1.72,0.431,0.331,48.7,0.527,16.3,0.485
AKRON,AKRON,52.7,7.5,0.518,0.435,0.319,0.696,0.222,-5.1,102.6,...,69.1,-6.82,8.4,-1.92,0.467,0.358,49.2,0.547,17.3,0.502
ALABAMA-AM,ALABAMA-AM,50.5,3.9,0.450,0.397,0.314,0.647,0.203,-23.2,88.1,...,68.3,-23.97,5.8,-8.04,0.354,0.303,48.2,0.480,20.9,0.448
ALABAMA-BIRMINGHAM,ALABAMA-BIRMINGHAM,59.3,11.6,0.545,0.488,0.291,0.750,0.218,10.1,109.8,...,69.5,4.90,7.7,-0.65,0.334,0.345,54.8,0.575,16.5,0.559


This section inputs all upsets from 2010-2018 and tests how the model performs

In [ ]:
game_list = [[2017,'MIDDLE-TENNESSEE','MINNESOTA','MIDDLE-TENNESSEE'],
[2016,'ARKANSAS-LITTLE-ROCK','PURDUE','ARKANSAS-LITTLE-ROCK'],
[2016,'YALE','BAYLOR','YALE'],
[2014,'STEPHEN-F-AUSTIN','VIRGINIA-COMMONWEALTH','STEPHEN-F-AUSTIN'],
[2014,'NORTH-DAKOTA-STATE','OKLAHOMA','NORTH-DAKOTA-STATE'],
[2014,'HARVARD','CINCINNATI','HARVARD'],
[2013,'MISSISSIPPI','WISCONSIN','MISSISSIPPI'],
[2013,'CALIFORNIA','NEVADA-LAS-VEGAS','CALIFORNIA'],
[2013,'OREGON','OKLAHOMA-STATE','OREGON'],
[2012,'SOUTH-FLORIDA','TEMPLE','SOUTH-FLORIDA'],
[2012,'VIRGINIA-COMMONWEALTH','WICHITA-STATE','VIRGINIA-COMMONWEALTH'],
[2011,'RICHMOND','VANDERBILT','RICHMOND'],
[2010,'CORNELL','TEMPLE','CORNELL'],
[2016,'NORTHERN-IOWA','TEXAS','NORTHERN-IOWA'],
[2016,'GONZAGA','SETON-HALL','GONZAGA'],
[2016,'WICHITA-STATE','ARIZONA','WICHITA-STATE'],
[2015,'DAYTON','PROVIDENCE','DAYTON'],
[2015,'UCLA','SOUTHERN-METHODIST','UCLA'],
[2014,'TENNESSEE','MASSACHUSETTS','TENNESSEE'],
[2014,'DAYTON','OHIO-STATE','DAYTON'],
[2013,'MINNESOTA','UCLA','MINNESOTA'],
[2016,'MIDDLE-TENNESSEE','MICHIGAN-STATE','MIDDLE-TENNESSEE'],
[2013,'FLORIDA-GULF-COAST','GEORGETOWN','FLORIDA-GULF-COAST'],
[2012,'LEHIGH','DUKE','LEHIGH'],
[2012,'NORFOLK-STATE','MISSOURI','NORFOLK-STATE'],
[2016,'STEPHEN-F-AUSTIN','WEST-VIRGINIA','STEPHEN-F-AUSTIN'],
[2015,'GEORGIA-STATE','BAYLOR','GEORGIA-STATE'],
[2015,'ALABAMA-BIRMINGHAM','IOWA-STATE','ALABAMA-BIRMINGHAM'],
[2014,'MERCER','DUKE','MERCER'],
[2013,'HARVARD','NEW-MEXICO','HARVARD'],
[2010,'OHIO','GEORGETOWN','OHIO'],
[2018,'MARSHALL','WICHITA-STATE','MARSHALL'],
[2018,'BUFFALO','ARIZONA','BUFFALO'],
[2016,'HAWAII','CALIFORNIA','HAWAII'],
[2013,'LA-SALLE','KANSAS-STATE','LA-SALLE'],
[2012,'OHIO','MICHIGAN','OHIO'],
[2011,'MOREHEAD-STATE','LOUISVILLE','MOREHEAD-STATE'],
[2010,'MURRAY-STATE','VANDERBILT','MURRAY-STATE'],
[2018,'LOYOLA-IL','MIAMI-FL','LOYOLA-IL'],
[2018,'SYRACUSE','TEXAS-CHRISTIAN','SYRACUSE'],
[2017,'RHODE-ISLAND','CREIGHTON','RHODE-ISLAND'],
[2017,'SOUTHERN-CALIFORNIA','SOUTHERN-METHODIST','SOUTHERN-CALIFORNIA'],
[2017,'XAVIER','MARYLAND','XAVIER']]
game_df_teams = pd.DataFrame(game_list)
game_df_teams.columns = ['Season','Away','Home','Actual Winner']
game_df_teams

In [ ]:
game_df = pd.DataFrame(game_list)
game_df.columns = ['Season','Away','Home','Actual Winner']
game_df = game_df[['Season','Away','Home']]
game_df = game_df.merge(team_data_abb, left_on = ['Season','Home'], right_on = ['Season','abbreviation'], how = 'left')
game_df = game_df.merge(team_data_abb, left_on = ['Season','Away'], right_on = ['Season','abbreviation'], how = 'left')
print(game_df)

In [ ]:
game_input = game_df.drop(columns = ['Season','Away','Home','abbreviation_x','abbreviation_y','Team_Id_x','Team_Id_y','win_percentage_x','win_percentage_y'], axis = 1)

In [ ]:
game_input

In [ ]:
y_pred_C = clf_C.predict(game_input)
y_pred_B = clf_B.predict(game_input)
y_pred_A = clf_A.predict(game_input)
Y_pred = clf.predict(game_input)

In [ ]:
print(y_pred_C,y_pred_B,y_pred_A,Y_pred)

In [ ]:
test = pd.DataFrame(Y_pred)

In [ ]:
games_df_out = game_df_teams.merge(test, left_index = True, right_index = True)
games_df_out.columns = ['Season','Away','Home','Actual Winner','Forecast']

In [ ]:
games_df_out['Winner'] = np.where(games_df_out['Forecast'] == 'Y', games_df_out['Away'], games_df_out['Home'])
games_df_out['Correct'] = np.where(games_df_out['Actual Winner'] == games_df_out['Winner'], "Correct!", "¯\_(ツ)_/¯")

In [ ]:
games_df_out[['Season','Away','Home','Actual Winner','Winner','Correct']]

This section is where you can input current year games and get an output.

In [205]:
first_round = [['NORTH-CAROLINA-CENTRAL','NORTH-DAKOTA-STATE',16,16],['DUKE','NORTH-DAKOTA-STATE',1,16],
               ['VIRGINIA-COMMONWEALTH','CENTRAL-FLORIDA',8,9],['MISSISSIPPI-STATE','LIBERTY',5,12],['VIRGINIA-TECH','SAINT-LOUIS',4,13],
               ['BELMONT','TEMPLE',11,11],['MARYLAND','TEMPLE',6,11] ,['LOUISIANA-STATE','YALE',3,14],['LOUISVILLE','MINNESOTA',7,10],
               ['MICHIGAN-STATE','BRADLEY',2,15],['FAIRLEIGH-DICKINSON','PRAIRIE-VIEW',16,16],['GONZAGA','PRAIRIE-VIEW',1,16],
               ['SYRACUSE','BAYLOR',8,9],['MARQUETTE','MURRAY-STATE',5,12],['FLORIDA-STATE','VERMONT',4,13],['ST-JOHNS-NY','ARIZONA-STATE',11,11],
               ['BUFFALO','ARIZONA-STATE',6,11],['TEXAS-TECH','NORTHERN-KENTUCKY',3,14],['NEVADA','FLORIDA',7,10],['MICHIGAN','MONTANA',2,15],
               ['VIRGINIA','GARDNER-WEBB',1,16],['MISSISSIPPI','OKLAHOMA',8,9],['WISCONSIN','OREGON',5,12], 
               ['KANSAS-STATE','CALIFORNIA-IRVINE',4,13],['VILLANOVA','SAINT-MARYS-CA',6,11], ['PURDUE','OLD-DOMINION',3,14],
               ['CINCINNATI','IOWA',7,10],['TENNESSEE','COLGATE',2,15],['NORTH-CAROLINA','IONA',1,16],['UTAH-STATE','WASHINGTON',8,9],
               ['AUBURN','NEW-MEXICO-STATE',5,12],['KANSAS','NORTHEASTERN',4,13],['IOWA-STATE','OHIO-STATE',6,11],['HOUSTON','GEORGIA-STATE',3,14],
               ['WOFFORD','SETON-HALL',7,10],['KENTUCKY','ABILENE-CHRISTIAN',1,16]]

second_round = [['DUKE','CENTRAL-FLORIDA',1,9],['MISSISSIPPI-STATE','VIRGINIA-TECH',5,4],['LOUISIANA-STATE','TEMPLE',3,11],
                  ['LOUISVILLE','MICHIGAN-STATE',7,2],['GONZAGA','BAYLOR',1,9],['MURRAY-STATE','FLORIDA-STATE',12,4],['ARIZONA-STATE','TEXAS-TECH',11,3]
                 ,['FLORIDA','MICHIGAN',10,2],['VIRGINIA','CALIFORNIA-IRVINE',1,13],['OREGON','CALIFORNIA-IRVINE',12,13],['VILLANOVA','PURDUE',6,3],
                 ['CINCINNATI','TENNESSEE',7,2],['NORTH-CAROLINA','WASHINGTON',1,9],['AUBURN','KANSAS',5,4],['OHIO-STATE','HOUSTON',11,3],
                 ['SETON-HALL','KENTUCKY',10,2]]

sweet_sixteen = [['DUKE','MISSISSIPPI-STATE',1,5],['LOUISIANA-STATE','MICHIGAN-STATE',3,2],['GONZAGA','FLORIDA-STATE',1,4],
                  ['TEXAS-TECH','MICHIGAN',3,2],['VIRGINIA','KANSAS-STATE',1,4],['TENNESSEE','VILLANOVA',2,6],['NORTH-CAROLINA','KANSAS',1,4]
                 ,['OHIO-STATE','KENTUCKY',11,2]]

elite_eight = [['DUKE','MICHIGAN-STATE',1,2],['GONZAGA','MICHIGAN',1,2],['VIRGINIA','VILLANOVA',1,6],
                  ['NORTH-CAROLINA','KENTUCKY',1,2]]

final_four = [['DUKE','MICHIGAN',1,2],['NORTH-CAROLINA','VILLANOVA',1,6]]

finals = [['DUKE','VILLANOVA',1,6]]


In [213]:
game_df_cur_team = pd.DataFrame(finals)
game_df_cur_team.columns = ['Away','Home','Away_Seed','Home_Seed']

In [214]:
game_df_cur = pd.DataFrame(finals)
game_df_cur.columns = ['Away','Home','Away_Seed','Home_Seed']
game_df_cur = game_df_cur[['Away','Home']]
game_df_cur = game_df_cur.merge(teams_df_19, left_on = ['Home'], right_on = ['abbreviation'], how = 'left')
game_df_cur = game_df_cur.merge(teams_df_19, left_on = ['Away'], right_on = ['abbreviation'], how = 'left')

In [215]:
game_input_cur = game_df_cur.drop(columns = ['Away','Home','abbreviation_x','abbreviation_y'], axis = 1)
game_input_cur = game_input_cur[['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y']]

In [216]:
cols = [['assist_percentage_x','block_percentage_x','effective_field_goal_percentage_x','field_goal_percentage_x','free_throw_attempt_rate_x','free_throw_percentage_x','net_rating_x','offensive_rating_x','offensive_rebound_percentage_x','pace_x','simple_rating_system_x','steal_percentage_x','strength_of_schedule_x','three_point_attempt_rate_x','three_point_field_goal_percentage_x','total_rebound_percentage_x','true_shooting_percentage_x','turnover_percentage_x','two_point_field_goal_percentage_x','assist_percentage_y','block_percentage_y','effective_field_goal_percentage_y','field_goal_percentage_y','free_throw_attempt_rate_y','free_throw_percentage_y','net_rating_y','offensive_rating_y','offensive_rebound_percentage_y','pace_y','simple_rating_system_y','steal_percentage_y','strength_of_schedule_y','three_point_attempt_rate_y','three_point_field_goal_percentage_y','total_rebound_percentage_y','true_shooting_percentage_y','turnover_percentage_y','two_point_field_goal_percentage_y']]
for col in cols:
    game_input_cur[col] = scale(game_input_cur[col])

In [217]:
out_19 = clf.predict(game_input_cur)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [218]:
temp = pd.DataFrame(out_19)
games_df_out_cur = game_df_cur_team.merge(temp, left_index = True, right_index = True)
games_df_out_cur.columns = ['Away','Home','Away_Seed','Home_Seed','Forecast']
games_df_out_cur['Winner'] = np.where(games_df_out_cur['Forecast'] == 'Y', games_df_out_cur['Away'], games_df_out_cur['Home'])

In [219]:
games_df_out_cur

,Away,Home,Away_Seed,Home_Seed,Forecast,Winner
0,DUKE,VILLANOVA,1,6,X,VILLANOVA


In [220]:
pd.Series(clf.feature_importances_, index = list(game_input_cur.columns.values)).sort_values(ascending = False)

simple_rating_system_x                 0.105991
strength_of_schedule_y                 0.096774
simple_rating_system_y                 0.082949
assist_percentage_y                    0.055300
strength_of_schedule_x                 0.050691
free_throw_attempt_rate_x              0.046083
net_rating_x                           0.041475
three_point_attempt_rate_y             0.036866
pace_x                                 0.036866
total_rebound_percentage_x             0.032258
two_point_field_goal_percentage_y      0.032258
offensive_rating_x                     0.027650
three_point_attempt_rate_x             0.027650
free_throw_attempt_rate_y              0.027650
total_rebound_percentage_y             0.027650
offensive_rating_y                     0.023041
turnover_percentage_x                  0.018433
assist_percentage_x                    0.018433
pace_y                                 0.018433
true_shooting_percentage_y             0.018433
net_rating_y                           0